In [ ]:

from utils import *
import community as community_louvain
plt.rcParams.update({'font.size': 17})
plt.figure(figsize=(6,5))

In [ ]:
G = nx.read_edgelist('../large_graph/com-dblp.ungraph.txt', delimiter='\t')
G.remove_edges_from(nx.selfloop_edges(G))
G.number_of_nodes(), G.number_of_edges()

mapping = {}
idx = 0
for v in G.nodes():
    mapping[v] = idx
    idx += 1
relabel_G = nx.relabel_nodes(G, mapping)

f = open('../large_graph/com-dblp.all.cmty.txt', 'r')
line = f.readline()
community = defaultdict(set)
communities = {}
com_idx = 0
while line:
    tmp = line.strip().split('\t')
#     if len(tmp)<5:
#         continue
    tmp_l = []
    for i in tmp:
        community[mapping[i]].add(com_idx)
        tmp_l.append(mapping[i])
    communities[com_idx] = tmp_l
    com_idx += 1
    line = f.readline()
    

## compare density histograms between all groundtruth communities and top 5k

In [ ]:

f = open('../large_graph/com-dblp.top5000.cmty.txt', 'r')
line = f.readline()
community5k = defaultdict(set)
communities5k = {}
com_idx = 0
while line:
    tmp = line.strip().split('\t')
#     if len(tmp)<5:
#         continue
    tmp_l = []
    for i in tmp:
        community5k[mapping[i]].add(com_idx)
        tmp_l.append(mapping[i])
    communities5k[com_idx] = tmp_l
    com_idx += 1
    line = f.readline()

In [ ]:
density = []
G = relabel_G
for cn in communities:
    c = communities[cn]
    if len(c)<2:
        continue
    d = 2*G.subgraph(c).number_of_edges()/(G.subgraph(c).number_of_nodes()*(G.subgraph(c).number_of_nodes()-1))
    density.append(d)
    
density5k = []
for cn in communities5k:
    c = communities5k[cn]
    d = 2*G.subgraph(c).number_of_edges()/(G.subgraph(c).number_of_nodes()*(G.subgraph(c).number_of_nodes()-1))
    density5k.append(d)

In [ ]:
a,b,c = plt.hist((density, density5k))

In [ ]:
d = a[0]/np.sum(a[0])
d5k = a[1]/np.sum(a[1])
xs = np.arange(0.05,1,0.1)

In [ ]:
plt.bar(xs-0.02, d, width=0.04, label='All communities')
plt.bar(xs+0.02, d5k, width=0.04, label='Top 5k communities')
plt.xlabel('density')
plt.ylabel('Empirical distribution')
# plt.legend()
plt.ylim(0,0.45)
plt.tight_layout()
plt.savefig('dblp_community_density_compare.png')
plt.show()

## Compute number of wedges and triangles contributed by each edge

In [ ]:
start_time = time.time()
for e in relabel_G.edges():
    relabel_G[e[0]][e[1]]['triangle'] = len(set(relabel_G[e[0]])&set(relabel_G[e[1]]))
print('triangle', time.time() - start_time)

start_time = time.time()
for e in relabel_G.edges():
    relabel_G[e[0]][e[1]]['wedge'] = len(set(relabel_G[e[0]])|set(relabel_G[e[1]])) - relabel_G[e[0]][e[1]]['triangle']
print('wedge', time.time() - start_time)

## motif-based community detection based on different similarity scores

In [ ]:

def plot(metrics,n, xticks=None, legend=True, savepath=None, highlight=None):
    plt.plot(np.array(metrics["num_components"])/n, 
             label='norm # CC', marker='d')
    plt.plot(np.array(metrics["edges_removed"]), label='norm edges', marker='x')
    plt.plot(np.array(metrics["largest_size"])/n, label='norm |largest CC|', marker='1')
    plt.plot(np.array(metrics["modularity"]), label='Modularity', marker='o')
    plt.plot(metrics["f1"], label='F1', marker='s')
    if legend:
        plt.legend(ncol=2,fontsize='14')
    if xticks is not None:
        xticks_vis = []
        for i in range(len(xticks)):
            if i%2==0:
                xticks_vis.append(round(xticks[i],3))
            else:
                xticks_vis.append('')
        plt.xticks([i for i in range(len(xticks))], xticks_vis, rotation=30)
    plt.ylim(-0.1,1.1)
    if highlight is not None:
        plt.axvspan(highlight[0], highlight[1], color='red', alpha=0.3)
    if savepath is not None:
        plt.tight_layout()
        plt.savefig(savepath)
    plt.show()

In [ ]:
def TW_edge_func(relabel_G, e, theta):
    return relabel_G[e[0]][e[1]]['triangle']-relabel_G[e[0]][e[1]]['wedge'] > theta

In [ ]:
TW_metrics = thresholding_by_partition(relabel_G, range(-30,0,2), TW_edge_func, communities, community)

In [ ]:
plot(TW_metrics, G.number_of_nodes(), range(-30,0,2), legend=True, savepath='dblp_tw.png', highlight=(8,10))

In [ ]:
tec_metrics = thresholding_by_partition(relabel_G, 0.01*np.arange(1, 30, 2), tec_edge_func, communities, community)

In [ ]:
plot(tec_metrics, G.number_of_nodes(), 0.01*np.arange(1, 30, 2), legend=False,
     savepath='dblp_tec.png', highlight=(7,9))

In [ ]:
k3_metrics = thresholding_by_partition(relabel_G, np.arange(0, 15), k3_edge_func, communities, community)

In [ ]:
plot(k3_metrics, G.number_of_nodes(), np.arange(0, 15), legend=False, savepath='dblp_k3.png', highlight=(4,6))

## Other competitors

### infomap

In [ ]:
import infomap
import networkx as nx

def infomap_community_detection(G, communities, community):
    """
    Apply the Infomap algorithm to detect communities in a network.

    Parameters:
    G (networkx.Graph): A NetworkX graph object.

    Returns:
    dict: A dictionary where keys are node IDs and values are community IDs.
    """

    # Create an Infomap instance
    im = infomap.Infomap(silent=True)

    # Add edges to the Infomap instance
    for edge in G.edges():
        im.add_link(*edge)

    # Run the Infomap clustering algorithm
    im.run()
    infox, infoy, infof1 = [], [], []
    for i in range(1,im.max_depth+1):
        info_com = im.get_modules(i)
        info_community = [[] for i in range(max(info_com.values())+1)]

        for i in info_com:
            info_community[info_com[i]].append(i)
        p,r,f1 = evaluate2(G, info_community, communities, community, False)
        infox.append(p)
        infoy.append(r)
        infof1.append(f1)
    return infox, infoy, infof1

In [ ]:
infox, infoy, infof1 = infomap_community_detection(relabel_G, communities, community)

### Louvain - resolution by dendrogram

In [ ]:
import community as community_louvain
louvainx = []
louvainy = []
total_t = []
# compute the best partition
start_time = time.time()
dendrogram = community_louvain.generate_dendrogram(relabel_G)
print('first period running', time.time() - start_time)
for j in range(len(dendrogram)):
    start_time = time.time()
    partition = community_louvain.partition_at_level(dendrogram, j)
    louvain_com = [[] for i in range(max(partition.values())+1)]
    
    for i in partition:
        louvain_com[partition[i]].append(i)
    total_t.append(time.time() - start_time)
    p,r,f1 = evaluate1(relabel_G, louvain_com, communities, False)
    louvainx.append(p)
    louvainy.append(r)
    print('precision and recall', p, r, f1)
print('avg running time', np.average(total_t))

### Louvain - resolution by parameter

In [ ]:
louvainx, louvainy, louvainf1, louvain_num = [], [], [], []
for resolution in [1e-6,1e-5,5e-5,1e-4,2e-4,5e-4,7e-4, 0.001,0.01]:
    partition = community_louvain.best_partition(relabel_G, resolution=resolution)
    louvain_com = [[] for i in range(max(partition.values())+1)]

    for i in partition:
        louvain_com[partition[i]].append(i)
    p,r,f1 = evaluate2(relabel_G, louvain_com, communities, community, False)
    louvainx.append(p)
    louvainy.append(r)
    louvainf1.append(f1)
    louvain_num.append(len(louvain_com))
    print(resolution, p,r,f1, len(louvain_com))

### label propagation communities

In [ ]:
lp_com = nx.algorithms.community.label_propagation_communities(relabel_G)
p,r,f1 = evaluate2(relabel_G, lp_com, communities, community, False)
lpc_x,lpc_y,lpc_f1,lpc_num = [p],[r],[f1],[len(lp_com)]
p,r,f1, len(lp_com)

### PCC and PMod code can be found here: https://github.com/jeshi96/parallel-correlation-clustering

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(6,5))

plt.plot(infox,infoy, marker='o', label='Infomap',c='pink')
plt.plot(lpc_x,lpc_y, marker='d', label='LPC',c='b')

# de-comment if you have results from pcc and pmod
# plt.plot(pcc_x, pcc_y, marker='>', label='PCC',c='purple')
# plt.plot(mod_x, mod_y, marker='x', label='PMod',c='brown')

plt.plot(TW_metrics['precision'],TW_metrics["recall"], marker='d', label='TW',c='r')
plt.plot(tec_metrics['precision'], tec_metrics["recall"], marker='>', label='Tectonic',c='g')
plt.plot(k3_metrics['precision'],k3_metrics["recall"], marker='x', label='K3',c='orange')

plt.legend()
plt.xlabel('precision')
plt.ylabel('recall')
# plt.tight_layout()
# plt.savefig('dblp.png')
plt.show()